# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.datasets import mnist
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras import backend as K
from keras.utils import np_utils


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [7]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.np_utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.np_utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [8]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11501568/11490434 [==============================] - 14s 1us/step


In [10]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [11]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [12]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [13]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [14]:
# Now, let's train our model on the digits 5,6,7,8,9
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 15s 64ms/step - loss: 1.6014 - accuracy: 0.2332 - val_loss: 1.5877 - val_accuracy: 0.3137
Epoch 2/5
230/230 [==============================] - 15s 65ms/step - loss: 1.5836 - accuracy: 0.2871 - val_loss: 1.5687 - val_accuracy: 0.4929
Epoch 3/5
230/230 [==============================] - 15s 65ms/step - loss: 1.5648 - accuracy: 0.3566 - val_loss: 1.5477 - val_accuracy: 0.6276
Epoch 4/5
230/230 [==============================] - 15s 64ms/step - loss: 1.5452 - accuracy: 0.4112 - val_loss: 1.5240 - val_accuracy: 0.6972
Epoch 5/5
230/230 [==============================] - 15s 65ms/step - loss: 1.5221 - accuracy: 0.4660 - val_loss: 1.4970 - val_accuracy: 0.7303
Training time: 0:01:14.684085
Test score: 1.4970004558563232
Test accuracy: 0.7303023934364319


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [15]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

In [17]:
feature_layers

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [18]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 5s 19ms/step - loss: 1.5812 - accuracy: 0.3144 - val_loss: 1.5582 - val_accuracy: 0.4721
Epoch 2/5
240/240 [==============================] - 5s 20ms/step - loss: 1.5504 - accuracy: 0.3848 - val_loss: 1.5232 - val_accuracy: 0.5719
Epoch 3/5
240/240 [==============================] - 5s 19ms/step - loss: 1.5185 - accuracy: 0.4522 - val_loss: 1.4883 - val_accuracy: 0.6772
Epoch 4/5
240/240 [==============================] - 5s 20ms/step - loss: 1.4869 - accuracy: 0.5171 - val_loss: 1.4536 - val_accuracy: 0.7527
Epoch 5/5
240/240 [==============================] - 5s 20ms/step - loss: 1.4543 - accuracy: 0.5743 - val_loss: 1.4193 - val_accuracy: 0.7978
Training time: 0:00:24.002909
Test score: 1.4193097352981567
Test accuracy: 0.7978205680847168


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [20]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_8 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_9 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [21]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 16s 64ms/step - loss: 1.6067 - accuracy: 0.2082 - val_loss: 1.5831 - val_accuracy: 0.2292
Epoch 2/5
240/240 [==============================] - 16s 65ms/step - loss: 1.5723 - accuracy: 0.2768 - val_loss: 1.5429 - val_accuracy: 0.4116
Epoch 3/5
240/240 [==============================] - 16s 66ms/step - loss: 1.5321 - accuracy: 0.3687 - val_loss: 1.4973 - val_accuracy: 0.6001
Epoch 4/5
240/240 [==============================] - 16s 65ms/step - loss: 1.4881 - accuracy: 0.4514 - val_loss: 1.4438 - val_accuracy: 0.7013
Epoch 5/5
240/240 [==============================] - 16s 65ms/step - loss: 1.4348 - accuracy: 0.5344 - val_loss: 1.3791 - val_accuracy: 0.7770
Training time: 0:01:18.453522
Test score: 1.3791168928146362
Test accuracy: 0.7769994139671326


In [22]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [23]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_8 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_9 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [24]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 5s 21ms/step - loss: 1.5878 - accuracy: 0.2631 - val_loss: 1.5549 - val_accuracy: 0.3491
Epoch 2/5
230/230 [==============================] - 5s 21ms/step - loss: 1.5543 - accuracy: 0.3054 - val_loss: 1.5193 - val_accuracy: 0.3872
Epoch 3/5
230/230 [==============================] - 5s 21ms/step - loss: 1.5223 - accuracy: 0.3538 - val_loss: 1.4833 - val_accuracy: 0.4653
Epoch 4/5
230/230 [==============================] - 5s 21ms/step - loss: 1.4881 - accuracy: 0.4114 - val_loss: 1.4468 - val_accuracy: 0.5787
Epoch 5/5
230/230 [==============================] - 5s 21ms/step - loss: 1.4534 - accuracy: 0.4704 - val_loss: 1.4098 - val_accuracy: 0.6711
Training time: 0:00:24.368235
Test score: 1.409836769104004
Test accuracy: 0.6710553169250488


---
### Machine Learning Foundation (C) 2020 IBM Corporation